# Proyecto 1

## Integrantes

* Maria Paula Gonzalez Escallon 
* Jessica Alejandra Robles Moreno
* Juan Esteban Vergara Ascencio

# Importar

In [1]:
# Librerías para manejo de datos
import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar
import numpy as np
np.random.seed(3301)
import pandas as pd
# Para preparar los datos
from sklearn.preprocessing import LabelEncoder
# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 
# Para usar KNN como clasificador
from sklearn.neighbors import KNeighborsClassifier
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
#Librerías para la visualización
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns 
from sklearn import tree

## Carga de Datos

In [4]:
df_info=pd.read_csv('SuicidiosProyecto.csv', sep=',', decimal=',', encoding = 'utf-8', index_col=0)

In [5]:
df_info.shape

(195700, 2)

In [6]:
df_info.head()

,text,class
Unnamed: 0,,
173271,i want to destroy myselffor once everything wa...,suicide
336321,I kinda got behind schedule with learning for ...,non-suicide
256637,I'm just not sure anymoreFirst and foremost: I...,suicide
303772,please give me a reason to liveThats too much ...,suicide
293747,27f struggling to find meaning moving forwardI...,suicide


In [7]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195700 entries, 173271 to 305170
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    195700 non-null  object
 1   class   195700 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB


In [8]:
df_info["class"].value_counts()

non-suicide    110165
suicide         85535
Name: class, dtype: int64

## Limpieza de datos

In [9]:
import string
punctuation = set(string.punctuation)
def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

In [10]:
df_info.head()["text"].apply(tokenize)

Unnamed: 0
173271    [i, want, to, destroy, myselffor, once, everyt...
336321    [i, kinda, got, behind, schedule, with, learni...
256637    [im, just, not, sure, anymorefirst, and, forem...
303772    [please, give, me, a, reason, to, livethats, t...
293747    [27f, struggling, to, find, meaning, moving, f...
Name: text, dtype: object

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
demo_vectorizer = CountVectorizer(
    tokenizer = tokenize,
    binary = True
)

In [12]:
from sklearn.model_selection import train_test_split
train_text, test_text, train_labels, test_labels = train_test_split(df_info["text"], df_info["class"], stratify=df_info["class"])
print(f"Training examples: {len(train_text)}, testing examples {len(test_text)}")

Training examples: 146775, testing examples 48925


In [13]:
real_vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)
train_X = real_vectorizer.fit_transform(train_text)
test_X = real_vectorizer.transform(test_text)

In [14]:
from sklearn.svm import LinearSVC
classifier = LinearSVC()
classifier.fit(train_X, train_labels)
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVC()

In [15]:
from sklearn.metrics import accuracy_score
predicciones = classifier.predict(test_X)
accuracy = accuracy_score(test_labels, predicciones)
print(f"Accuracy: {accuracy:.4%}")

Accuracy: 93.4410%


## Clasificacion de textos

In [17]:
df_test=pd.read_csv('SuicidiosProyectoSinEtiqueta.csv', sep=',', decimal=',', encoding = 'utf-8', index_col=0)

In [18]:
df_test.shape

(10300, 1)

In [19]:
df_test.head()

,text
Unnamed: 0,
66529,"Suicide Thoughts Getting StrongerOK, since the..."
293637,"Guys, I just asked my crush... if she wanted t..."
198884,I'm going to kill myselfOf course I'm not but ...
186897,I did it! I slept facing the wall.\n\nOverall ...
89024,I don't want to be dead...I want to be alive a...


In [20]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10300 entries, 66529 to 20052
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    10300 non-null  object
dtypes: object(1)
memory usage: 160.9+ KB


### Clasificacion

In [21]:
frases_X = real_vectorizer.transform(df_test["text"])
predicciones = classifier.predict(frases_X)

In [23]:
for text, label in zip(df_test["text"],predicciones):
  print(f"{label:5} - {text}")

Streaming output truncated to the last 5000 lines.
I can’t explain how my feelings go from 0-10 in a  matter of an hour.
It’s all physical the mental pain turned physical telling my brain I’m I can’t believe your doing this again.
I’m disconnected and self reliant.
Feeling abandoned in a room full of people.
Why are you here
Nobody likes you 
We don’t connect 
Anxiety taking over.
Your friends aren’t really your friends and they think your annoying, everyone is just playing nice because they like your other friend more.
Why are you acting weird 
Stop distancing yourself
No wonder you’re alone 
No one wonder your ugly/fat/disgusting 
It’s all your fault
Suicide is right, it’ll all feel better. The bridge is close and it’s all so easy. 
Those pills taste good and it’s all for the taking.
I can’t breathe 
I want to die 
All of the time and I’m constantly falling 
Felling alone and being alone are completely different but I have both and somewhat crave it.
This isnt about me you narcissist

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




suicide - When does therapy cure anhedonia and if it doesnt why shouldn't I suicide?I have drug induced anhedonia. 

I use therapy constantly and look for an effect. It has had 0 effects and then makes me suicidal. Im not going to live with this condition and I want to kill myself. My life is over.

Therapy: "No your life is not over"

Check feelings: symptoms are the same even if I replace that thought.

Same thought occurs again due to the symptoms

Therapy: Exercise

-exercises-

Check symptoms: symptoms are identical to before

No matter what I do the symptoms are the same. Therefore I just want to commit suicide.

Im not able to enjoy or feel things anymore. I cannot talk to people anymore at all.

Why shouldnt I suicide? Its not going to get better tomorrow or the day after....

And I do not have the patience to wait.

I am making plans for getting ECT perhaps and if that doesnt work then ill suicide. 
suicide - I wish I had never tried fasting It’s the closest I’ve been to an e